COMP9418 Assignment 1
Bryan Liauw

Task 1 codes

In [ ]:
from random import randint
from random import randrange
import matplotlib.pyplot as plt

In [ ]:
class Node:
    def __init__(self,name):
        self.name = name
        self.in_node = []
        self.out_node = []
        self.neighbour = []

This is the class for Nodes. neighbour is basically for in and out node specifically for the path finding part of the algorithm.

In [ ]:
def add_edge(node1,node2):
    node1.out_node.append(node2)
    node2.in_node.append(node1)
    node1.neighbour.append(node2)
    node2.neighbour.append(node1)
    return
def remove_edge(node1,node2):
    node1.out_node.remove(node2)
    node2.in_node.remove(node1)
    node1.neighbour.remove(node2)
    node2.neighbour.remove(node1)
    return
def remove_node(node):
    for i in node.in_node:
        i.out_node.remove(node)
    for i in node.out_node:
        i.in_node.remove(node)
    for i in node.neighbour:
        i.node.neighbour.remove(node)
    return

Several helper functions to remove and add edges and nodes.

In [ ]:
def random_dag(nodes, edges):
    G = {}
    for i in range(nodes):
        G[chr(97+i)] = Node(chr(97+i))
    while edges > 0:
        a = chr(97+randint(0,nodes-1))
        b=a
        while b==a:
            b = chr(97+randint(0,nodes-1))
        add_edge(G[a],G[b])
        edges -=1
    return G

Generate a random DAG with nodes and edges

In [ ]:
def find_path(x,y,G):
    visited = []
    stack = [x]
    while stack:
        node = stack.pop()
        if node not in visited:
            if node== y:
                return True
            visited.append(node)
            for i in G[node].neighbour:
                stack.append(i.name)
    return False

Function to find path in a graph

In [ ]:
def d_separation(G,X,Y,Z):
    leafs = [g for g in G if len(G[g].out_node)==0 and len(G[g].in_node)>=1]
    print(leafs)

    for x in X:
        if x in leafs:
            leafs.remove(x)
    for y in Y:
        if y in leafs:
            leafs.remove(y)
    for z in Z:
        if z in leafs:
            leafs.remove(z)
    for i in leafs:
        remove_node(G[i])
    for z in Z:
        temp = G[z].out_node
        for i in temp:
            remove_edge(G[z],i)
    for x in X:
        for y in Y:
            if find_path(x,y,G):
                return False
    return True

The main part of the algorithm: removal of edges and leaf nodes before finding a path

Task 2: Estimate Bayesian Network parameters from data

In [ ]:
import networkx as nx
from random import randint
from random import randrange
import matplotlib.pyplot as plt
import csv
import pandas as pd
from collections import Counter

In [ ]:
def create_dag():
    G = nx.DiGraph()
    G.add_node('Age')
    G.add_node('Location')
    G.add_node('BreastDensity')
    G.add_node('Size')
    G.add_node('LymphNodes')
    G.add_node('Metastasis')
    G.add_node('BC')
    G.add_node('Mass')
    G.add_node('Shape')
    G.add_node('MC')
    G.add_node('AD')
    G.add_node('Margin')
    G.add_node('SkinRetract')
    G.add_node('FibrTissueDev')
    G.add_node('NippleDischarge')
    G.add_node('Spiculation')

    G.add_edge('Age','BC')
    G.add_edge('Location','BC')
    G.add_edge('BreastDensity','Mass')
    G.add_edge('BC','Mass')
    G.add_edge('BC','Metastasis')
    G.add_edge('BC','MC')
    G.add_edge('BC','SkinRetract')
    G.add_edge('BC','NippleDischarge')
    G.add_edge('BC','AD')
    G.add_edge('Mass','Size')
    G.add_edge('Mass','Shape')
    G.add_edge('Mass','Margin')
    G.add_edge('AD','FibrTissueDev')
    G.add_edge('FibrTissueDev','SkinRetract')
    G.add_edge('FibrTissueDev','Spiculation')
    G.add_edge('FibrTissueDev','NippleDischarge')
    G.add_edge('Spiculation','Margin')
    G.add_edge('Metastasis','LymphNodes')

    return G

I could've used a dictionary or something but I originally intended to use the networkx package. It seems that it is unnecessary to remove it since it doesn't interfere with much anyways.

In [ ]:
def d_separation(G,file):
    #print(nx.adjacency_matrix(G).todense())
    data = pd.read_csv(file)
    probTable = {}
    outcomeSpace = {}
    #print(data)
    for i in G.nodes:
        if i != 'None':
            probTable[i] = Counter(list(data[i]))
            probTable[i] = dict(probTable[i])
            for j in probTable[i]:
                probTable[i][j] = float(probTable[i][j])/20000
            outcomeSpace[i] = tuple(probTable[i].keys())
    print(probTable)
    print(outcomeSpace)
    #nx.draw(G,with_labels= True)
    #plt.show()

Generate a probability table and outcome space.

Task 3 codes

In [ ]:
from random import randint
from random import randrange
from random import random
import csv
import copy
import pandas as pd
from collections import Counter
class Node:
    def __init__(self,name):
        self.name = name
        self.in_node = []
        self.out_node = []
        self.neighbour = []
        self.prob = {}
    def given(self):
        in_node_names = [i.name for i in self.in_node]
        return in_node_names
    def outnodes(self):
        in_node_names = [i.name for i in self.out_node]
        return in_node_names
def add_edge(node1,node2):
    node1.out_node.append(node2)
    node2.in_node.append(node1)
    node1.neighbour.append(node2)
    node2.neighbour.append(node1)

def remove_edge(node1,node2):
    node1.out_node.remove(node2)
    node2.in_node.remove(node1)
    node1.neighbour.remove(node2)
    node2.neighbour.remove(node1)

def remove_node(node):
    for i in node.in_node:
        i.out_node.remove(node)
    for i in node.out_node:
        i.in_node.remove(node)
    return


Modified nodes class from task 1 codes but generally reusing the same code

In [ ]:
def generate_dag():
    G = {}
    G['Age'] = Node('Age')
    G['Location'] = Node('Location')
    G['BreastDensity']= Node('BreastDensity')
    G['Size'] = Node('Size')
    G['LymphNodes'] = Node('LymphNodes')
    G['Metastasis']= Node('Metastasis')
    G['BC'] = Node('BC')
    G['Mass'] = Node('Mass')
    G['Shape'] = Node('Shape')
    G['MC'] = Node('MC')
    G['AD'] = Node('AD')
    G['Margin'] = Node('Margin')
    G['SkinRetract'] = Node('SkinRetract')
    G['FibrTissueDev'] = Node('FibrTissueDev')
    G['NippleDischarge']= Node('NippleDischarge')
    G['Spiculation']= Node('Spiculation')

    add_edge(G['Age'],G['BC'])
    add_edge(G['Location'],G['BC'])
    add_edge(G['BreastDensity'],G['Mass'])
    add_edge(G['BC'],G['Mass'])
    add_edge(G['BC'],G['Metastasis'])
    add_edge(G['BC'],G['MC'])
    add_edge(G['BC'],G['SkinRetract'])
    add_edge(G['BC'],G['NippleDischarge'])
    add_edge(G['BC'],G['AD'])
    add_edge(G['Mass'],G['Size'])
    add_edge(G['Mass'],G['Shape'])
    add_edge(G['Mass'],G['Margin'])
    add_edge(G['AD'],G['FibrTissueDev'])
    add_edge(G['FibrTissueDev'],G['SkinRetract'])
    add_edge(G['FibrTissueDev'],G['Spiculation'])
    add_edge(G['FibrTissueDev'],G['NippleDischarge'])
    add_edge(G['Spiculation'],G['Margin'])
    add_edge(G['Metastasis'],G['LymphNodes'])
    return G

Generate DAG based on the graph given

In [ ]:
def topological_sort(G,root):
    visited = []
    stack = list(root)
    while stack:
        node = stack.pop()
        visited.append(node)
        while G[node].out_node:
            tempnode = G[node].out_node[0]
            remove_edge(G[node],tempnode)
            if len(tempnode.in_node) == 0:
                stack.append(tempnode.name)
    return visited

Topological sort for getting the orders of traversal

In [ ]:
def sampling(G,file):
    data = pd.read_csv(file)
    roots = [g for g in G if len(G[g].out_node)>0 and len(G[g].in_node)==0]
    order = topological_sort(copy.deepcopy(G),roots)

    for i in range(data.shape[0]):
        for j in data:
            if len(G[j].in_node) == 0:
                if data[j][i] not in G[j].prob:
                    G[j].prob[data[j][i]]=0
                G[j].prob[data[j][i]]+=1
            else:
                tempkey = []
                for k in G[j].given():
                    tempkey.append(data[k][i]) 
                if ",".join(tempkey) not in G[j].prob:
                    G[j].prob[",".join(tempkey)] = {}
                if data[j][i] not in G[j].prob[",".join(tempkey)]:
                    G[j].prob[",".join(tempkey)][data[j][i]]=0
                G[j].prob[",".join(tempkey)][data[j][i]]+=1
    for i in G:
        if i in roots:
            temp = sum(G[i].prob.values())
            for j in G[i].prob:
                G[i].prob[j] = float(G[i].prob[j])/temp
        else:
            for j in G[i].prob:
                temp = sum(G[i].prob[j].values())
                for k in G[i].prob[j]:
                    G[i].prob[j][k] = float(G[i].prob[j][k])/temp

    sampledata =[]

    for i in range(1000):
        tempdata = {}
        for j in order:
            temp = random()
            if j in roots:
                for k in G[j].prob:
                    temp -= G[j].prob[k]
                    if temp <=0.:
                        tempdata[j] = k
                        break
            else:
                tempgiven = []
                for g in G[j].given():
                    tempgiven.append(tempdata[g])
                for k in G[j].prob[",".join(tempgiven)]:
                    temp -= G[j].prob[",".join(tempgiven)][k]
                    if temp <=0.:
                        tempdata[j] = k
                        break
        sampledata.append(tempdata)

    return sampledata

Get the CSV file then generate the probability model stored in individual nodes. Using these probabilities, create sample data by subtracting from the random generated number

Task 4 codes

In [2]:
from random import randint
from random import randrange
from random import random
import matplotlib.pyplot as plt
import csv
import copy
import pandas as pd
from collections import Counter
from sklearn.neighbors import KNeighborsClassifier


class Node:
    def __init__(self,name):
        self.name = name
        self.in_node = []
        self.out_node = []
        self.neighbour = []
        self.prob = {}
        self.sum = {}
    def given(self):
        in_node_names = [i.name for i in self.in_node]
        return in_node_names
    def outnodes(self):
        in_node_names = [i.name for i in self.out_node]
        return in_node_names
def add_edge(node1,node2):
    node1.out_node.append(node2)
    node2.in_node.append(node1)
    node1.neighbour.append(node2)
    node2.neighbour.append(node1)

def remove_edge(node1,node2):
    node1.out_node.remove(node2)
    node2.in_node.remove(node1)
    node1.neighbour.remove(node2)
    node2.neighbour.remove(node1)

def remove_node(node):
    for i in node.in_node:
        i.out_node.remove(node)
    for i in node.out_node:
        i.in_node.remove(node)
    return


def generate_dag():
    G = {}
    G['Age'] = Node('Age')
    G['Location'] = Node('Location')
    G['BreastDensity']= Node('BreastDensity')
    G['Size'] = Node('Size')
    G['LymphNodes'] = Node('LymphNodes')
    G['Metastasis']= Node('Metastasis')
    G['BC'] = Node('BC')
    G['Mass'] = Node('Mass')
    G['Shape'] = Node('Shape')
    G['MC'] = Node('MC')
    G['AD'] = Node('AD')
    G['Margin'] = Node('Margin')
    G['SkinRetract'] = Node('SkinRetract')
    G['FibrTissueDev'] = Node('FibrTissueDev')
    G['NippleDischarge']= Node('NippleDischarge')
    G['Spiculation']= Node('Spiculation')

    add_edge(G['Age'],G['BC'])
    add_edge(G['Location'],G['BC'])
    add_edge(G['BreastDensity'],G['Mass'])
    add_edge(G['BC'],G['Mass'])
    add_edge(G['BC'],G['Metastasis'])
    add_edge(G['BC'],G['MC'])
    add_edge(G['BC'],G['SkinRetract'])
    add_edge(G['BC'],G['NippleDischarge'])
    add_edge(G['BC'],G['AD'])
    add_edge(G['Mass'],G['Size'])
    add_edge(G['Mass'],G['Shape'])
    add_edge(G['Mass'],G['Margin'])
    add_edge(G['AD'],G['FibrTissueDev'])
    add_edge(G['FibrTissueDev'],G['SkinRetract'])
    add_edge(G['FibrTissueDev'],G['Spiculation'])
    add_edge(G['FibrTissueDev'],G['NippleDischarge'])
    add_edge(G['Spiculation'],G['Margin'])
    add_edge(G['Metastasis'],G['LymphNodes'])
    return G

def topological_sort(G,root):
    visited = []
    stack = list(root)
    while stack:
        node = stack.pop()
        visited.append(node)
        while G[node].out_node:
            tempnode = G[node].out_node[0]
            remove_edge(G[node],tempnode)
            if len(tempnode.in_node) == 0:
                stack.append(tempnode.name)
    return visited

def sampling(G,file):
    data = pd.read_csv(file)
    roots = [g for g in G if len(G[g].out_node)>0 and len(G[g].in_node)==0]
    order = topological_sort(copy.deepcopy(G),roots)

    for i in range(data.shape[0]):
        for j in data:
            if len(G[j].in_node) == 0:
                if data[j][i] not in G[j].prob:
                    G[j].prob[data[j][i]]=0
                G[j].prob[data[j][i]]+=1
            else:
                tempkey = []
                for k in G[j].given():
                    tempkey.append(data[k][i]) 
                if ",".join(tempkey) not in G[j].prob:
                    G[j].prob[",".join(tempkey)] = {}
                if data[j][i] not in G[j].prob[",".join(tempkey)]:
                    G[j].prob[",".join(tempkey)][data[j][i]]=0
                G[j].prob[",".join(tempkey)][data[j][i]]+=1
    for i in G:
        if i in roots:
            temp = sum(G[i].prob.values())
            G[i].sum["sum"] = temp
            for j in G[i].prob:
                G[i].prob[j] = float(G[i].prob[j])/temp
        else:
            for j in G[i].prob:
                temp = sum(G[i].prob[j].values())
                G[i].sum[j] = temp
                for k in G[i].prob[j]:
                    G[i].prob[j][k] = float(G[i].prob[j][k])/temp

    sampledata =[]

    for i in range(1000):
        tempdata = {}
        for j in order:
            temp = random()
            if j in roots:
                for k in G[j].prob:
                    temp -= G[j].prob[k]
                    if temp <=0:
                        tempdata[j] = k
                        break
            else:
                tempgiven = []
                for g in G[j].given():
                    tempgiven.append(tempdata[g])
                for k in G[j].prob[",".join(tempgiven)]:
                    temp -= G[j].prob[",".join(tempgiven)][k]
                    if temp <=0:
                        tempdata[j] = k
                        break
        sampledata.append(tempdata)

    return G, order, sampledata

Basically the same as the task 3 codes

In [ ]:
def predict(G,order,sampledata):
    final_pred = 0
    bc = ""
    for i in ['No','Invasive','Insitu']:
        pred = 1
        for g in order:
            tempgiven = []
            tempattr = sampledata[g]
            if g == "BC":
                tempattr = i
            for j in G[g].given():
                if j == "BC":
                    tempgiven.append(i)
                else:
                    tempgiven.append(sampledata[j])
            if tempgiven: 
                if tempattr not in G[g].prob[",".join(tempgiven)]:
                    pred = pred * 1/(G[g].sum[",".join(tempgiven)]+1+len(G[g].prob[",".join(tempgiven)]))
                else:
                    pred = pred*G[g].prob[",".join(tempgiven)][tempattr]
            else:
                pred = pred* G[g].prob[tempattr]
        if final_pred< pred:
            final_pred = pred
            bc = i
    return bc

Create prediction by finding the maximum joint distribution given the outcomes.

In [ ]:
data = pd.read_csv("bc 2.csv")
for i in data:
    data[i] = data[i].astype('category')
cat_columns = data.select_dtypes(['category']).columns
data[cat_columns] = data[cat_columns].apply(lambda x: x.cat.codes)
target = data['BC']
del data['BC']
clf=KNeighborsClassifier(n_neighbors = 100)
clf.fit(data,target)
pred = clf.predict(data)
correct = 0
for i in range(len(pred)):
    if pred[i] == target[i]:
        correct+=1
print(float(correct)/len(a))

K nearest neighbour implementations. Required to change the data from string to category in order to change it into nunmbers